## Hangman

- Make a list that keeps 40 words
- Read a file to pick 40 words and put them in the list
- Pick a word randomly for each hangman game
- Show placeholders for an answer
- Show placeholders for errors
- Use queue to hold error characters
- Show a character in its correct location if user inputs a character in the answer word
- Show how many chances are left
- Pop up a window if the game ends
- Game Over: Retry or Exit (Iterate the game if user presses ‘retry’)
- Close the app if user presses ‘quit’
- Show #trials and #wins
- Username
- Password 
- Store records in a file
- Store records in a DB
- Show the ranks with #wins/#trials using tuple and dictionary after reading records in the file or DB
- Use modules
- Use functions
- Use classes
- Make comments
- Use casting
- Use error handling
- Make it pretty

In [7]:
# 필수 라이브러리--------------------------------------------------------------------#

from tkinter import *
import random
from tkinter import messagebox
from tkinter import Menu
import mysql.connector
import module
import pandas as pd

#시작할 때 DB에 접근합니다.----------------------------------------------------------------#
host = ""
port = 3306
user = ""
password = ""
database = "Hangman"

try: # 없으면 만들어줍니다.
    mydb = mysql.connector.connect(host = host,
                                   port = port,
                                   user = user,
                                   password = password,
                                   charset='utf8')
    # DB와 테이블 제작.
    mycursor = mydb.cursor()
    mycursor.execute("CREATE DATABASE Hangman")
    mycursor.execute("USE Hangman")
    mycursor.execute("""CREATE TABLE hm_score (ID varchar(30) NOT NULL,
                                             PASSWORD varchar(30) NOT NULL,
                                             TRIALS int default 0,
                                             WIN int default 0,
                                             WIN_RATE float default 0,
                                             CONSTRAINT hm_score_PK PRIMARY KEY (id))""")

except: # 이미 있으면 접속을 합니다.
    mydb = mysql.connector.connect(host = host,
                                   port = port,
                                   user = user,
                                   password = password,
                                   database = database)
    mycursor = mydb.cursor()
    print("Use Hangman")
#-------------------------------------------------------------------------------------#

#--메인입니다.--------------------------------------------------------------------------#
class Hangman:
    def __init__(self, master): # 마스터 페이지 영역입니다.
        self.master = master
        master.title("Hangman")
        self.master.geometry('500x300')
        
        # 마스터 페이지 텍스트
        self.l1 = Label(master, text=" ").pack()
        self.l2 = Label(master, text="Welcome to the Hangman World", font=("Arial Bold", 15)).pack()
        self.l3 = Label(master, text="Guess the Python Methods/Functions", font=(12)).pack()
        self.l4 = Label(master, text=" ").pack()
        
        # 마스터 페이지 버튼(로그인, 사인, 스코어)
        self.b1 = Button(master, text="Login", width = 20, bg="green", font=("Arial Bold", 10), command=self.login).pack()
        self.b2 = Button(master, text="Sign in", width = 20, bg="orange", font=("Arial Bold", 10), command=self.sign).pack()
        self.b3 = Button(master, text="Score", width = 20, bg="purple", font=("Arial Bold", 10), command=self.score).pack()
        self.b4 = Button(master, text="Quit", width = 20, bg="red", font=("Arial Bold", 10), command=self.master.destroy).pack()
        
        # 제작자 코멘트
        self.m1 = Label(master, text=" ").pack()
        self.m2 = Label(master, text=" ").pack()
        self.m3 = Button(master, text="Info", width = 10, bg="white", command=self.info).pack()
        
        
    def info(self):
        messagebox.showwarning("Info","MAKER: thinpig99")
        
        
    def game(self): # 게임 영역입니다.
        # 단어 셋팅
        self.words = open(r"./python_words.txt", 'r')
        self.words_list = self.words.readlines() #한줄에 한 단어씩 txt 파일에서 한 줄(\n 포함)씩 읽은 내용을 lines 리스트에 포함
        self.word = random.choice(self.words_list).upper()[:-1]
        
        # 게임 기본 셋팅.
        self.word_completion = "ㅡ"* len(self.word) # 이건 밑줄.
        self.guessed = False # 정답 여부
        self.guessed_letters = [] # 스펠링 창고
        self.guessed_words = [] # 단어 창고
        self.tries = 6 # 목숨

        # 여기가 게임 화면.
        self.page_game = Tk()
        self.page_game.title("Game")
        self.page_game.geometry('800x500')
        
        # 상단 개인 점수판
        self.game_l10 = Label(self.page_game, text="ID: " + str(self.user_name), font=("Arial Bold", 12))
        self.game_l11 = Label(self.page_game, text="Trials: " + str(self.user_trials), font=("Arial", 12))
        self.game_l12 = Label(self.page_game, text="Wins: " + str(self.user_win), font=("Arial", 12))
        
        # 하단 게임 화면
        self.game_l1 = Label(self.page_game, text=" ", font=("Arial Bold", 15))
        self.game_l2 = Label(self.page_game, text="Let's Guess the Word", font=("Arial Bold", 15)) # 여백
        self.game_l3 = Label(self.page_game, text="Remain Chance(s) : " + str(self.tries), fg="red")# 기회
        self.game_l4 = Label(self.page_game, text="Already Guessed Letters : " + str(self.guessed_letters)) # 틀린 스펠링
        self.game_l5 = Label(self.page_game, text="Already Guessed Words : " + str(self.guessed_words)) # 틀린 단어
        self.game_l6 = Label(self.page_game, text=self.word_completion) # 워드 컴프리션
        self.game_l7 = Label(self.page_game, text=" ") # 여백
        self.game_e1 = Entry(self.page_game, text="", textvariable=StringVar(), state=NORMAL)
        self.game_l8 = Label(self.page_game, text=" ", font=("Arial Bold", 20))
        self.game_b1 = Button(self.page_game, text="Guess", width = 10, bg="purple", font=("Arial Bold", 10), command = self.guess) # 추측 버튼
        self.game_l9 = Label(self.page_game, text=" ") # 여백
        self.game_b2 = Button(self.page_game, text="Reset", width=10, bg="orange", font=("Arial Bold", 10), command= self.reset) # 다시 하기
        self.game_b3 = Button(self.page_game, text="Quit", width=10, bg="red", font=("Arial Bold", 10), command=self.page_game.destroy) # 나가기
        
        # 상단 추가 패킹
        self.game_l10.pack()
        self.game_l11.pack()
        self.game_l12.pack()
        
        # .get()을 받으려면 패킹을 따로 해줘야 한다.
        self.game_l1.pack()
        self.game_l2.pack()
        self.game_l3.pack()
        self.game_l4.pack()
        self.game_l5.pack()
        self.game_l6.pack()
        self.game_l7.pack()
        self.game_e1.pack()
        self.game_l8.pack()
        self.game_b1.pack()
        self.game_l9.pack()
        self.game_b2.pack()
        self.game_b3.pack()
        
        
    def guess(self): # 게임창 게스 버튼
        self.guess = self.game_e1.get().upper() # 입력값으로부터 받아 온다.
        
        # 스펠링이라면?
        if len(self.guess) == 1 and self.guess.isalpha(): # isalpha()를 통해 특수문자나 공백을 받지 않는다.
            
            # 이미 있는 스펠링이라면?
            if self.guess in self.guessed_letters:
                self.game_l8['text'] = self.guess + " have already guessed!" # 이거 바꿔주고.
            
            # 단어 안에 없는 스펠링이라면?
            elif self.guess not in self.word:
                self.game_l8['text'] = self.guess + "  is not in the word" # 틀렸다고 해주고
                self.tries -= 1 # 목숨 깎고
                self.game_l3['text'] = "Remain Chance(s) : " + str(self.tries) # 표시창 업데이트
                self.guessed_letters.append(self.guess) # 스펠링 창고에 넣고
                self.guessed_letters.sort() # 소팅 해줍니다.
                self.game_l4['text'] = "Already Guessed Letters : " + str(self.guessed_letters) # 표시창 업데이트
                
            # 단어 안에 있다면?
            else:
                self.game_l8['text'] = "Good job! "+self.guess+" is in the word!" # 맞췄다고 해주고
                self.guessed_letters.append(self.guess) # 스펠링 창고에 넣습니다.
                self.guessed_letters.sort() # 소팅 해줍니다.
                self.game_l4['text'] = "Already Guessed Letters : " + str(self.guessed_letters)
                
                # 그리고 워드 컴플리션을 바꿔줘야 합니다.
                word_as_list = list(self.word_completion) # 문제를 리스트로 뽑아냅니다.
                indices = [i for i, letter in enumerate(self.word) if letter == self.guess]# 그리고 스펠의 인덱스를 뽑는다.
                # 순서를 찾았으면 바꿔줍니다.
                for index in indices:
                    word_as_list[index] = self.guess
                self.word_completion = "".join(word_as_list)
                
                # 빈칸이 하나도 안 남게 되면? 정답이겠죠?
                if "ㅡ" not in self.word_completion:
                    self.guessed = True
        
        # 단어라면?
        elif len(self.guess) > 1 and self.guess.isalpha():
            
            # 이미 추측한 단어를 다시 말하면?
            if self.guess in self.guessed_words:
                self.game_l8['text'] = "You already guessed the word " + self.guess
            
            # 정답이 아니면?
            elif self.guess != self.word:
                self.tries -= 1 # 목숨 깎고
                self.guessed_words.append(self.guess) # 단어 창고에 넣습니다.
                self.guessed_words.sort() # 소팅 해줍니다.
                self.game_l8['text'] = self.guess+" is not the word."
                self.game_l3['text'] = "Remain Chance(s) : " + str(self.tries)
                self.game_l5['text'] = "Already Guessed Words : " + str(self.guessed_words)
            
            # 정답이 맞으면?
            else:
                self.guessed = True
                self.word_completion = self.word
                
        # 스펠링도 단어도 아닌 이상한 걸 넣었다면?
        else:
            self.game_l8['text'] = "Not a valid guess."
            
        # 게임 종료 조건
        if self.tries == 0: # 목숨이 다 달아나면?
            self.game_e1['state'] = DISABLED
            self.game_b1['state'] = DISABLED
            self.game_l8['text'] = "The word was " + self.word
            messagebox.showwarning('That is to bad','Sorry, you ran out of tries.')
        elif self.guessed: # guessed 가 True가 되면, 즉 정답을 맞추면.
            # 게임창 셋팅
            self.game_e1['state'] = DISABLED
            self.game_b1['state'] = DISABLED
            self.game_l8['text'] = "You win! It was " + self.word
            
            # 정보 업데이트
            self.user_win += 1 # 승수를 하나 올려줍니다.
            self.game_l12['text'] = "Wins: " + str(self.user_win)# 표시창도 업데이트
            # DB 적재
            sql = "UPDATE hm_score SET win = %s WHERE id = %s"
            val = (self.user_win, self.user_name)
            mycursor.execute(sql, val)
            
            messagebox.showwarning('Nice Play','You guessed the word!')
        else: # 아니면 그냥 지나치슈.
            pass
        
        self.game_l6['text'] = self.word_completion # 다시 밑줄 표시
        self.game_e1.delete(0,END) # 엔트리 창 리셋
        
        
    def reset(self): # 게임창 리셋 버튼
        
        # 단어 다시 불러오기
        self.words = open(r"./python_words.txt", 'r')
        self.words_list = self.words.readlines() #한줄에 한 단어씩 txt 파일에서 한 줄(\n 포함)씩 읽은 내용을 lines 리스트에 포함
        self.word = random.choice(self.words_list).upper()[:-1]
        
        # 상태 리셋
        self.word_completion = "ㅡ"* len(self.word) # 이건 밑줄.
        self.guessed = False # 정답 여부
        self.guessed_letters = [] # 스펠링 창고
        self.guessed_words = [] # 단어 창고
        self.tries = 6 # 목숨
        
        # 개인 이력 업데이트
        self.user_trials += 1 # 판수를 하나 늘려줍니다.
        self.game_l11['text'] ="Trials: " + str(self.user_trials) # 표시창도 업데이트.    
        # DB 적재
        sql = "UPDATE hm_score SET trials = %s WHERE id = %s"
        val = (self.user_trials, self.user_name)
        mycursor.execute(sql, val)
        
        # 표시창 리셋
        self.game_l3['text'] = "Remain Chance(s) : " + str(self.tries)
        self.game_l4['text'] = "Already Guessed Letters : " + str(self.guessed_letters)
        self.game_l5['text'] = "Already Guessed Words : " + str(self.guessed_words)
        self.game_l6['text'] = self.word_completion
        self.game_l8['text'] = " "
        
        # 입력창 오픈
        self.game_e1['state'] = NORMAL
        self.game_b1['state'] = NORMAL
        self.game_e1.delete(0,END) # 엔트리 창 리셋
    
    
    def login(self): # 로그인 영역입니다.
        self.page_login = Tk()
        self.page_login.title("Login")
        self.page_login.geometry('250x200')
    
        self.login_l1 = Label(self.page_login, text="ID")
        self.login_e1 = Entry(self.page_login)
        self.login_l2 = Label(self.page_login, text="PW")
        self.login_e2 = Entry(self.page_login)
        self.login_l3 = Label(self.page_login, text=" ")
        self.login_b1 = Button(self.page_login, text="Login", bg="green", width = 10, font=("Arial Bold", 10), command=self.login_to_game)
        self.login_b2 = Button(self.page_login, text="Cancel", bg="red", width = 10, font=("Arial Bold", 10), command=self.page_login.destroy)
        
        # 아이디 비번 .get()으로 받기 위한 패킹
        self.login_l1.pack()
        self.login_e1.pack()
        self.login_l2.pack()
        self.login_e2.pack()
        self.login_l3.pack()
        self.login_b1.pack()    
        self.login_b2.pack()
        
        
    def login_to_game(self): # 로그인 하고 게임으로 진입
        # 아이디랑 비번 받고
        name = self.login_e1.get()
        pw = self.login_e2.get()
        
        if len(name) < 4 or len(pw) < 4: # 너무 짧으면 깝니다.
            messagebox.showwarning('Warning','ID and PW must longer than 4')
            
        else: #길게 입력하면 일단 입장 시킵니다.
            # ID 존재하는지 확인
            mycursor.execute("SELECT * FROM hm_score")
            id_list = mycursor.fetchall()
            name_check = False # 존재여부입니다. True여야 로그인을 시킬 겁니다.
            if not name_check: # id 체크가 True면
                for k, y, t, w, r in id_list: # id_list로부터 정보를 받습니다.
                    if name != k: # id 가 없으면
                        pass # 그냥 지나칩니다.
                    else: # id가 있으면
                        db_pw = y # 패스워드를 받습니다.
                        name_check = True # 바꿔줍니다.
                        break;
                
                # ID 존재확인 했으면 PW확인합니다.
                if name_check: # for문을 돌았는데도 True 라면 로그인 합니다.
                    if pw != y: # 그런데 pw가 틀립니다.
                            messagebox.showwarning('Warning','Wrong Password!!!')
                    else: # pw도 맞습니다.
                            # 그렇다면 개인 정보 끌어옵니다.
                        self.user_name = k
                        self.user_trials = t+1 # 판수는 로그인하며 자동으로 하나 늘어납니다.
                        self.user_win = w
                        
                        # DB 적재
                        sql = "UPDATE hm_score SET trials = %s WHERE id = %s"
                        val = (self.user_trials, self.user_name)
                        mycursor.execute(sql, val)
                        
                        # 로그인 창 닫고 게임 창으로 진입
                        self.page_login.destroy()
                        self.game()
                else: # 위 for문에서 False로 바꼈다면 ID가 없는 것입니다.
                    messagebox.showwarning('Warning','Wrong ID or Does not Exist!!!')

                    
    def sign(self): # 사인 영역입니다.
        self.page_sign = Tk()
        self.page_sign.title("sign")
        self.page_sign.geometry('250x200')
    
        self.sign_l1 = Label(self.page_sign, text="ID")
        self.sign_e1 = Entry(self.page_sign)
        self.sign_l2 = Label(self.page_sign, text="PW")
        self.sign_e2 = Entry(self.page_sign)
        self.sign_l3 = Label(self.page_sign, text=" ")
        self.sign_b1 = Button(self.page_sign, text="Sign In", bg="green", width = 10, font=("Arial Bold", 10), command=self.sign_to_login)
        self.sign_b2 = Button(self.page_sign, text="Cancel", bg="red", width = 10, font=("Arial Bold", 10), command=self.page_sign.destroy)

        self.sign_l1.pack()
        self.sign_e1.pack()
        self.sign_l2.pack()
        self.sign_e2.pack()
        self.sign_l3.pack()
        self.sign_b1.pack()
        self.sign_b2.pack()
        
        
    def sign_to_login(self): # 가입하고 메인으로 복귀
        # 아이디랑 비번 받고
        new_id = self.sign_e1.get()
        new_pw = self.sign_e2.get()
        
        if len(new_id) < 4 or len(new_pw) < 4: # 너무 짧으면 깝니다.
            messagebox.showwarning('Warning','ID and PW must longer than 4')
            
        else: #길게 입력하면 일단 입장 시킵니다. DB를 인덱싱해서 중복여부에 따라 가입시킵니다.
            mycursor.execute("SELECT id FROM hm_score") #DB를 끌어옵니다.
            id_list = mycursor.fetchall()
            
            name_dupl = True # 중복여부입니다. True 가입을 시킬 겁니다.
            
            if name_dupl: # 인덱싱을 위해서 들어갑니다.
                # 중복 여부를 파악해줍니다.
                for i in range(len(id_list)): #인덱싱합니다.
                    if id_list[i][0] != new_id: # 만약 중복 값이 없으면
                        pass # 지나칩니다.
                    else: # 중복값이 있으면
                        name_dupl = False #False로 바꿔줍니다.
                        break; # 포문을 멈추게 하고 밑으로 내립니다.
                        
                # 중복 확인이 끝나면 가입시킵니다.
                if name_dupl: # 여전히 True면 중복 값이 없다는 소리니 
                    sql = "INSERT INTO hm_score (id, password) VALUES (%s, %s)"
                    val = (new_id, new_pw)
                    mycursor.execute(sql, val)
                    messagebox.showwarning('OK','Now you can enjoy!')
                    self.page_sign.destroy() # 가입이 끝났으니 가입창을 종료합니다.
                else: # False로 바꼈으면 중복이 있다는 말입니다.
                    messagebox.showwarning('Warning','This id is already exist!!!')
               
            
    def score(self): # 스코어 영역입니다.
        self.page_score = Tk()
        self.page_score.title("score")
        self.page_score.geometry('500x250')
        
        # 헤드에 들어갈 텍스트를 만들어줍니다.
        head = ["Rank", "ID", "Trials", "Wins", "Win-Rate"]
        
        # 스코어는 데이터 프레임으로 받겠습니다.
        mycursor.execute("SELECT id, trials, win, win_rate FROM hm_score")
        myresult = mycursor.fetchall()
        score_db = pd.DataFrame(myresult)
        
        # 스코어창 테이블을 만듭니다.
        for row in range(8): #세로 8줄. 7등까지만 보여줍니다.
            for column in range(5): # 가로 5줄.
                if row==0: # 첫 번째 줄은 헤드입니다.
                    label = Label(self.page_score, bg="white", fg="black")
                    label['text'] = head[column]
                    label.config(font=('Arial bold', 12))
                    label.grid(row=row, column=column, sticky="nsew")
                    self.page_score.grid_columnconfigure(column, weight=1)
                else: # 나머지는 스코어입니다.
                    if column==0: # 첫 번째 column에는 랭크를 반환합니다.
                        label = Label(self.page_score, bg="black", fg="white", padx=3, pady=3)
                        label['text'] = row
                        label.grid(row=row, column=column, sticky="nsew", padx=1, pady=1)
                        self.page_score.grid_columnconfigure(column, weight=1)
                    else: # 나머지는 스코어를 받습니다.
                        label = Label(self.page_score, bg="black", fg="white", padx=3, pady=3)
                        label['text'] = score_db[column-1][row-1]
                        label.grid(row=row, column=column, sticky="nsew", padx=1, pady=1)
                        self.page_score.grid_columnconfigure(column, weight=1)
                    
        # 스코어창 나가기 버튼
        self.score_b1 = Button(self.page_score, text="Cancel", bg="red", width = 10, font=("Arial Bold", 10), command=self.page_score.destroy)
        self.score_b1.grid(row=8, column=2)

        
root = Tk()
wang = Hangman(root)
root.mainloop()

Use Hangman
